- trf
    - ca37: distance=passthrough
    - fa15: distance=pwr
- imp
    - 2474: tune=False
    - 20cf: tune=True
- clf
    - 2c44: ensemble=True, time=20
    - 25f2: ensemble=False, time=20

In [12]:
from LiveAMP import *
# , train_code=202408
self = AMP(cycle_day=126, styp_code='n', crse_code='ansc1319', param=cartesian(param_dcts)[0])
# self = run_amp(126)
self.get_X()
self.get_summary()
self.rslt[' 50%'].disp(1000)
# self = get_stack(126)


In [8]:
self.mlt.loc[('n',202408)]

,regstr,admit,census,regstr_pct,admit_pct,mlt
crse_code,,,,,,
_allcrse,1026,1026,1053,97.435897,97.435897,1.026316
acct2301,12,12,12,100.0,100.0,1.0
acct2302,1,1,2,50.0,50.0,2.0
acom1310,8,8,8,100.0,100.0,1.0
acom2301,1,1,1,100.0,100.0,1.0
...,...,...,...,...,...,...
univ0314,81,81,81,100.0,100.0,1.0
univ0324,2,2,3,66.666667,66.666667,1.5
univ0332,4,4,4,100.0,100.0,1.0


In [ ]:
self.mlt

In [ ]:
self.summary

In [ ]:
clf_dct = self.param['clf'][2] | {'task':'classification', 'verbose':0, 'time_budget':5}
g = lambda y: y.query(f"crse_code=='{self.crse_code}'").droplevel('crse_code')
Z_proc = (
    self.X_proc
    .join(  self.y['credit']) .fillna({'credit':0})
    .join(g(self.y['regstr'])).fillna({'regstr':False})
    .join(g(self.y['actual'])).fillna({'actual':False})
    .addlevel({'crse_code':self.crse_code, 'clf_hash':self.param['clf'][0]})
    .prep(bool=True, cat=True)
)
Z = Z_proc.query(f"pred_code!={self.proj_code}").copy()
G = Z.query(f"sim==0").groupby(['pred_code','actual','__coll_code'], observed=True).ngroup().rename('grp').to_frame().rsindex('idx')

In [ ]:
Z.groupby(['pred_code','sim']).filter(lambda x:x['actual'].sum()>0).vc('pred_code')

In [ ]:
def train(self, train_code='all'):
    X = Z.copy() if train_code == self.proj_code else Z.query(f"pred_code=={train_code}").copy()
    y = X.pop('actual')
    idx = G.groupby('grp').sample(frac=0.75, random_state=clf_dct['seed']).index
    msk = X.eval('idx.isin(@idx)')
    dct = clf_dct | {
        'X_train':X[msk],
        'y_train':y[msk],
        'X_val':X[~msk],
        'y_val':y[~msk],
    }
    self.clf = fl.AutoML(**dct)
    self.clf.fit(**dct)
    self.Y = Z_proc[['actual']].assign(proba=self.clf.predict_proba(Z_proc.drop(columns='actual'))[:,1]).prep(bool=True).copy()
    self.train_score = self.clf.best_result['val_loss'] * 100
    self.clf = self.clf._trained_estimator


In [ ]:
X.vc('pred_code')

In [ ]:
G = Z.query(f"sim==0").groupby(['pred_code','actual','__coll_code'], observed=True).ngroup().rename('grp').to_frame().rsindex('idx')

In [ ]:
idx = G.groupby('grp').sample(frac=0.75, random_state=clf_dct['seed']).index
trn = Z.query(f'idx.isin(@idx)')
tst = Z.query(f'not idx.isin(@idx)')
Z.shape, trn.shape, tst.shape

In [ ]:
Z = Z.rename_axis(index={'index':'idx'})

In [ ]:
t = G.groupby('grp').sample(frac=0.75, random_state=clf_dct['seed']).reset_index()['index']
Z.query("idx==1602")
# Z.reset_index()['index'].isin(t)

In [ ]:
trn, tst = train_test_split(X, y)#, stratify=g, train_size=0.75, random_state=clf_dct['seed'])

In [ ]:
# def train(self, train_code=None):
dct = clf_dct | {
    'X_train':X_trn,
    'y_train':y_trn,
    # 'eval_method':'cv',
    # 'n_splits':5,
    # 'split_type':'stratified',
    'X_val':X_tst,
    'y_val':y_tst,
    # 'sample_weight':1+(2*y[mask]-1)*wgt,
    # 'sample_weight_val':1+(2*y-1)*wgt,
    # 'log_file_name': self.path.with_stem(f"{self.path.stem}{iter}").with_suffix('.log'),
}
self.clf = fl.AutoML(**dct)
self.clf.fit(**dct)
self.Y = Z_proc[['actual']].assign(proba=self.clf.predict_proba(Z_proc.drop(columns='actual'))[:,1]).prep(bool=True).copy()
# return self
# train(self)

In [ ]:
X = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt3/126/X.pkl')
X['mlt'].query("crse_code=='easc2310'")

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt3/126/summary/n/easc2310/fa15/1bfb/1414.pkl')
w['summary']


In [ ]:
self.report

In [ ]:
self.cycle_date

In [ ]:
read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/126/summary/n/anth2351/fa15/1bfb/4a31.pkl').keys()

In [ ]:
g = lambda y: y.query(f"crse_code=='{self.crse_code}'").droplevel('crse_code')
Z = (
    self.X_proc
    # .join(  self.y['credit']) .fillna({'credit':0})
    # .join(g(self.y['regstr'])).fillna({'regstr':False})
    .join(g(self.y['actual'])).fillna({'actual':False})
    # .addlevel({'crse_code':self.crse_code, 'train_code':self.train_code, 'clf_hash':self.param['clf'][0]})
    .addlevel({'crse_code':self.crse_code, 'clf_hash':self.param['clf'][0]})
    .prep(bool=True, cat=True)
)
Z = Z[Z.eval(f"pred_code=={self.proj_code}") | (Z.groupby(['pred_code','sim'])['actual'].transform('sum')>=10)]

In [ ]:
# Z.rsindex(['pred_code','sim']).groupby(['pred_code','sim'])['actual'].sum().sort_index().disp(1000)
A = Z.query(f"pred_code!={self.proj_code} & sim==0").groupby('pred_code')['actual'].sum().query('actual>600').index.tolist()
A
# A[A>600].index

In [ ]:
Z[Z.eval(f"pred_code=={self.proj_code}") | (Z.groupby(['pred_code','sim'])['actual'].transform('sum')>=10)]
Z[mask]
# Z['ct'] = Z.groupby(['pred_code','sim'])['actual'].transform('sum')
# Z.query("pred_code==202408 | ct >= 100000")

#['actual'].sum()

In [ ]:
dct = {'a':2,'b':1}
min(dct, key=dct.get)

In [ ]:
x = np.array([-678.938727,-69.417566,-47.140036])
np.abs(np.sum(x))*10+np.mean(np.abs(x))

In [ ]:
from LiveAMP import *
self = run_amp(126)

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/126/Y/n/_allcrse/202308/fa15/1bfb/2dbb.pkl')
Y = w['Y']
Y['proba'] = Y['actual'] == True
log_loss(Y['actual'], Y['proba'])
clf = fl.AutoML()


In [ ]:
from flaml import AutoML
w = AutoML()
w.fit?

In [ ]:
from flaml import AutoML
from sklearn.datasets import load_iris

# Initialize an AutoML instance

# Specify automl goal and constraint
automl_settings = {
    "time_budget": 10,  # in seconds
    "metric": "log_loss",
    # "metric": "accuracy",
    "task": "classification",
    # "log_file_name": "iris.log",
    'verbose':0,
}
clf = AutoML(**automl_settings)
X_train, y_train = load_iris(return_X_y=True)
# Train with labeled input data
clf.fit(X_train=X_train, y_train=y_train, **automl_settings)
# Predict
# print(clf.predict_proba(X_train))
# Print the best model
# print(clf.model.estimator)


In [ ]:
from sklearn.metrics import log_loss, accuracy_score
log_loss(y_train, clf.predict_proba(X_train)), accuracy_score(y_train, clf.predict(X_train))
# clf.score(X_train, y_train)

In [ ]:
import plotly.express as px
from LiveAMP import *
w = []
for path in sorted(AMP().root_path.parent.iterdir()):
    for fn in (path / 'summary/n/_allcrse').rglob('*.pkl'):
        w.append(read(fn)['rslt'][' 50%'].addlevel({'cycle_day':path.stem}))
df = pd.concat(w).query('pred_code==202308').reset_index()
fig = px.line(
    df,
    x='cycle_day',
    y='202408_projection',
    color='train_code',
    # text="cycle_day",
    symbol="train_code",
    )
fig.update_layout(hovermode="x unified")
fig.show()

In [ ]:
self.y['adm'].disp()

In [ ]:
# y = {k:y.query(f"crse_code=='{self.crse_code}'").droplevel('crse_code') for k,y in self.y.items()}
self.y['credit']

In [ ]:
self.y['actual'].disp()

In [ ]:
self.y['admit'].disp()

In [ ]:
self.credit_hr
self.y[self.cycle_day]

In [ ]:
# y = self.y.query(f"crse_code=='{self.crse_code}'").droplevel('crse_code').rename(self.crse_code)
y = self.y.query(f"crse_code=='{self.crse_code}'").unstack()
cols = [self.crse_code]
y = self.y.query(f"crse_code.isin({cols})").unstack()#.droplevel(0,1)

y.disp()
# Z = (
#     self.X
# # #     # .join(self.credit_hr)
#     .join(y)
# # #     # .fillna({'credit_hr':0})
# # #     # .fillna({c: False for c in cols})
# # #     # .addlevel({'crse_code':self.crse_code, 'train_code':self.train_code, 'clf_hash':self.param['clf'][0]})
# # #     # .rsindex(['index','crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim'])
# # #     # .prep(bool=True, cat=True)
# )
# Z.disp()

In [ ]:
self.y.vc('enrolled')
# self.credit_hr.disp()

In [ ]:
# M = self.mlt.copy()
# M['reg_pct'] = M['reg'] / M['end'] * 100
# M['adm_pct'] = M['adm'] / M['end'] * 100
self.mlt.query("styp_code=='n' & crse_code.isin(['_allcrse','biol1406'])").reset_index().sort_values(['crse_code','pred_code']).disp(100)#crse_code=='_allcrse'")

In [ ]:
y = self.y.query("crse_code.str.contains('biol1406')").unstack().droplevel(0,1)
z = y
z = self.X[[]].join(y)
A = z.groupby('pred_code').sum()
A['mlt'] = A.iloc[:,0] / A.iloc[:,1]
A['2024'] = A['mlt'] * A.iloc[-1,1]
A

In [ ]:
self.mlt.query("crse_code=='biol1406'")

In [ ]:
# self.crse_code = '_allcrse'
cols = [self.crse_code+'_cur', self.crse_code]
y = self.y.query(f"crse_code.isin({cols})").unstack().droplevel(0,1)[cols]
Z = (
    self.X[[]]#_proc
    # .join(self.credit_hr)
    .join(y)
    # .fillna({'credit_hr':0})
    # .fillna({c: False for c in cols})
    # .addlevel({'crse_code':self.crse_code, 'train_code':self.train_code, 'clf_hash':self.param['clf'][0]})
    # .rsindex(['index','crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim'])
    # .prep(bool=True, cat=True)
)
# Z.disp()
Z.groupby('pred_code').sum()

In [ ]:
Z.iloc[:,-2:].groupby('pred_code').sum()

In [ ]:
self.mlt

In [ ]:
Z.disp(4)

In [ ]:
self.X_proc

In [ ]:
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/126/summary/n/_allcrse/202208/fa15/1bfb/2dbb.pkl'
w = read(fn)
w['rslt'][' 50%']

In [ ]:
self.credit_hr

In [ ]:
self.mlt.query("crse_code.isin(['_allcrse','biol1406'])")

In [ ]:
133/7
126/7

In [ ]:
terms[133][202408]

In [ ]:
qry = "select A.current_date, count(*) as ct from opeir.admissions_fall2024 A group by A.current_date order by A.current_date asc"
db.execute(qry).disp(1000)

In [ ]:
from LiveAMP import *
T = Term(term_code=202308, cycle_day=133, show='adm', overwrite='adm')
T.get_adm()

In [ ]:
T.adm.disp()

In [ ]:
np.arange(126, 200, 7)

In [ ]:
from LiveAMP import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/126/X.pkl'
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/126/X.pkl'
# fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/126/summary/n/_allcrse/202308/fa15/1bfb/2dbb.pkl'
w = read(fn)
# w['rslt'][' 50%']
w['mlt'].query("crse_code=='biol1406'")

In [ ]:
self.mlt

In [ ]:
w['X'].missing().disp(1000)

In [ ]:
from LiveAMP import *
# Flags().run()
for cycle_week in range(19, 19+10):
    cycle_day=cycle_week*7
    print(cycle_day)
    run_amp(cycle_day=cycle_day)

# for cycle_day in range(132, 128, -1):
# for cycle_day in [137,130]:
#     print(cycle_day)
#     A = AMP(cycle_day=cycle_day)
#     A.print_days()
# A.get_X()
# run_amp(126)
# self = get_stack(130, '_2024_05_10')
# self.get_Y()
# self.get_summary()
# self.summary
# # self.Y
# getsizeof(self.__dict__)

In [ ]:
trm = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/trm.parq')
trm.query("term_code==202108")

In [ ]:
4-6

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/130/Y/n/_allcrse/202308/fa15/1bfb/2dbb.pkl')
getsizeof(w)

In [ ]:
w['Y']

In [ ]:
# w = pd.to_datetime('today').normalize()-5
# w.day_of_week

# (pd.Timestamp.now().day-1)#.day_of_week
pd.to_datetime('2024-05-13').day_name()

In [ ]:
self = AMP()
str(self.root_path).replace('data','1234')

In [ ]:
read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/130/summary/n/_allcrse/202208/fa15/1bfb/2dbb.pkl').keys()

In [ ]:
dct = {'a':1,'b':2}
pd.Series(dct).sort_values(ascending=False)

In [ ]:
w = AMP().load('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/131/Y/n/ansc1119/202208/fa15/1bfb/4add.pkl', force=True)

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt2/133/summary/n/_allcrse/202208/fa15/1bfb/2dbb.pkl')
w
# getsizeof(w)
# w['Y']

In [ ]:
L=[1,2]
L.remove(1)
L

In [ ]:
w['Y'].memory_usage().sum()

In [ ]:
w['Y']

In [ ]:
def getsizeof(x):
    if isinstance(x, dict):
        for k,v in x.items():
            print(k, size(v))
    else:
        return sys.getsizeof(x)
size(w)

In [ ]:
w = read('/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/131/Y/n/arts1304/202108/fa15/1bfb/4add.pkl')

w.keys()
# w['rslt'][' 50%'].memory_usage().sum()
# w['Y']

In [ ]:
w['Y'].rsindex('index').info()
w['Y'].info()

In [ ]:
from LiveAMP import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/131/Y/n/_allcrse/202208/fa15/1bfb/4add.pkl'
self = AMP().load(path=fn)
self.summarize()
self.__dict__.keys()

In [ ]:
from LiveAMP import *
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/131/Y/n/ansc1119/202108/fa15/1bfb/4add.pkl'
self = AMP().load(path=fn, force=True)
# self.train_score = 0
# self.get_X()
# self.y
# def summarize(self):
#     self.get_X()
#     self.get_Y()
#     grp = ['index','crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim']
#     self.Y = self.Y.rsindex(grp)
#     grp.remove('index')
#     S = self.Y.groupby(grp).apply(lambda y: pd.Series({
#         'actual': y['actual'].sum(),
#         # 'predicted': y['predicted'].sum(),
#         'predicted': y['proba'].sum(),
#         'train_score': self.train_score,
#         'test_score': log_loss(y['actual'], y['proba'], labels=[False,True], sample_weight=1+(2*y['actual']-1)*self.weight) * 100,
#         'weight': self.weight,
#     }))
#     S['predicted'] *= S['actual'] > 0
#     proj_mask = S.eval(f"pred_code==@self.proj_code")
#     proj_col = f'{self.proj_code}_projection'
#     S = (
#         S[~proj_mask]
#         .join(S[proj_mask]['predicted'].droplevel('pred_code').rename(proj_col))
#         .join(self.mlt)
#         .sort_index()
#     ).prep().fillna(0)
    # for k in ['predicted','actual',proj_col]:
    #     S[k] *= S['mlt']
    # S[proj_col+'_pct_change'] = (S[proj_col] / S.groupby(S.index.names.difference({'pred_code'})).transform('last')['actual'] - 1) * 100
    # alpha = 1
    # S['overall_score'] = (S['train_score'] + alpha * S['test_score']) / (1 + alpha)
    # S['error'] = S['predicted'] - S['actual']
    # S['error_pct'] = S['error'] / S['actual'] * 100
    # S.loc[S.eval('predicted==0'), 'predicted':] = pd.NA
    # self.summary = S[[proj_col,proj_col+'_pct_change','predicted','actual','error','error_pct','mlt','overall_score','test_score','train_score','weight']].prep()#.fillna(0)
    # grp.remove('sim')
    # self.rslt = {str(stat): self.summary.groupby(grp).agg(stat) for stat in self.stats}
    # self.rslt[' 50%'].disp(100)
    # for k in ['X_proc','y','mlt']:
    #     del self[k]
# summarize(self)

In [ ]:
fn = '/home/scook/institutional_data_analytics/admitted_matriculation_projection/resources/rslt/131/Y/n/ansc1119/202108/fa15/1bfb/4add.pkl'
self = AMP().load(path=fn, force=True)
self.train_score = 0
def summarize(self):
    S = self.Y.groupby(['crse_code','levl_code','styp_code','train_code','pred_code','trf_hash','imp_hash','clf_hash','sim']).apply(lambda y: pd.Series({
        'actual': y['actual'].sum(),
        # 'predicted': y['predicted'].sum(),
        'predicted': y['proba'].sum(),
        'train_score': self.train_score,
        'test_score': log_loss(y['actual'], y['proba'], labels=[False,True], sample_weight=1+(2*y['actual']-1)*self.weight) * 100,
        'weight': self.weight,
    }))
    proj_mask = S.eval(f"pred_code==@self.proj_code")
    proj_col = f'{self.proj_code}_projection'
    S = (
        S[~proj_mask]
        .join(S[proj_mask]['predicted'].droplevel('pred_code').rename(proj_col))
        .join(self.mlt)
        .sort_index()
    ).prep().fillna(0)
    for k in ['predicted','actual',proj_col]:
        S[k] *= S['mlt']
    S[proj_col+'_pct_change'] = (S[proj_col] / S.groupby(S.index.names.difference({'pred_code'})).transform('last')['actual'] - 1) * 100
    alpha = 1
    S['overall_score'] = (S['train_score'] + alpha * S['test_score']) / (1 + alpha)
    S['error'] = S['predicted'] - S['actual']
    S['error_pct'] = S['error'] / S['actual'] * 100
    S.loc[S.eval('predicted==0'), 'predicted':] = pd.NA
    self.summary = S[[proj_col,proj_col+'_pct_change','predicted','actual','error','error_pct','mlt','overall_score','test_score','train_score','weight']].prep()#.fillna(0)
    grp = [k for k in self.summary.index.names if k!= 'sim']
    self.rslt = {str(stat): self.summary.groupby(grp).agg(stat) for stat in self.stats}
    self.rslt[' 50%'].disp(100)
    return self.summary
summarize(self)

In [ ]:
self.mlt

In [ ]:
self.mlt.query("crse_code=='ansc1119'")

In [ ]:
sp.optiminze.show_options('minimize_scalar', method='bounded')

In [ ]:
self.mlt.query("crse_code.isin(['chem1407','kine2315','kine2319','kine3319','fina1360'])").sort_values(['styp_code','crse_code','pred_code']).reset_index()

In [ ]:
self.mlt.query("crse_code=='chem1407'").prep()